In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib widget

In [2]:
df_cab = pd.read_csv('cab_rides.csv').dropna()
df_cab['time_stamp'] = pd.to_datetime(df_cab['time_stamp'], unit='ms')

In [3]:
df_cab = df_cab.sort_values('time_stamp')

In [4]:
df_weather = pd.read_csv('weather.csv')
df_weather['time_stamp'] = pd.to_datetime(df_weather['time_stamp'], unit='s')
df_weather['rain'] = df_weather['rain'].fillna(0)
df_weather = df_weather.sort_values('time_stamp')

In [5]:
df = pd.merge_asof(df_cab, df_weather, on='time_stamp')

In [6]:
features = ['cab_type', 'name', 'temp', 'clouds', 'pressure', 'rain', 'humidity', 'wind']
df = df[features]
df.head()

,cab_type,name,temp,clouds,pressure,rain,humidity,wind
0,Lyft,Lux Black XL,41.02,0.87,1014.4,0.0,0.92,1.59
1,Uber,Black,41.02,0.87,1014.4,0.0,0.92,1.59
2,Lyft,Lyft,41.02,0.87,1014.4,0.0,0.92,1.59
3,Uber,UberXL,41.02,0.87,1014.4,0.0,0.92,1.59
4,Uber,UberX,41.02,0.87,1014.4,0.0,0.92,1.59


In [7]:
for col in df.columns:
    if df[col].dtype == 'O':
        mapping = {item: i for i, item in enumerate(df[col].unique())}
        df[col] = df[col].map(mapping)

In [8]:
df.head()

,cab_type,name,temp,clouds,pressure,rain,humidity,wind
0,0,0,41.02,0.87,1014.4,0.0,0.92,1.59
1,1,1,41.02,0.87,1014.4,0.0,0.92,1.59
2,0,2,41.02,0.87,1014.4,0.0,0.92,1.59
3,1,3,41.02,0.87,1014.4,0.0,0.92,1.59
4,1,4,41.02,0.87,1014.4,0.0,0.92,1.59
